## Process all the data using TPP

In [33]:
# TODO: Delete this cell before release
# #Change the names of all the fastq files to SRA numbers for testing
# from shutil import copyfile
# import pandas as pd

# samp_df = pd.read_csv('input/Sample_Labels.csv')
# for uid, sra in zip(samp_df['Run_UID'],samp_df['SRA Accession']):
#     f1 = 'input/'+uid + '_1.fastq'
#     f1_out = 'input/'+sra + '_1.fastq'
#     f2 = 'input/'+uid + '_2.fastq'
#     f2_out = 'input/'+sra + '_2.fastq'
    
#     copyfile(f1, f1_out, follow_symlinks=False)
#     copyfile(f2, f2_out, follow_symlinks=False)

In [ ]:
%%bash
#SET THESE VARIABLES FOR YOUR LOCAL FILE STRUCTURE:
PATH_TO_DATA=input #Note: Any fastq files located here will be processed for alignment with tpp
OUT_DIR=output
REFS=input/Genome/MAC109.fa
GENBANK_FILE=input/Genome/MAC109.gb

#Definition of variables for processing
PYTHON2=$(which python2)
BWA=$(which bwa)
BWA_ALG="aln"

REPLICON_ID="CP029332,CP029333,CP029334"
FASTQ_DIR=$PATH_TO_DATA

PREFIXES_OUTFILE=$OUT_DIR/`basename $FASTQ_DIR`_prefixes.txt

# These are used for creating a CSV file
CSV_OUTFILE=$OUT_DIR/`basename $FASTQ_DIR`.csv
UNIQUE_FIELDS="locus_tag"
FIELDS="product regulatory_class bound_moiety"

#Parameter settings for tpp
PRIMER=AACCTGTTA
MISMATCHES=2
WINDOW_SIZE=6

###################################################################

#Process raw fastq files using tpp
COUNTER=0
INITIAL_START_TIME=$SECONDS
for FASTQ in $FASTQ_DIR/*_1.fastq; do
#for FASTQ in $FASTQ_DIR/*53606_1.fastq; do
    echo "******** Run $COUNTER: $FASTQ ********"
    READS1=$FASTQ
    READS2=${FASTQ/_1.fastq/_2.fastq}

    OUTNAME=$(basename $FASTQ)
    OUTNAME=${OUTNAME/_1.fastq/}
    tpp -himar1 -bwa $BWA -bwa-alg $BWA_ALG -ref $REFS -replicon-ids $REPLICON_ID -reads1 $READS1 -reads2 $READS2 -window-size $WINDOW_SIZE -primer $PRIMER -mismatches $MISMATCHES -output $OUT_DIR/$OUTNAME &
done
wait

echo "Creating prefixes file with all prefixes from all runs..."
basename -a $OUT_DIR/*.wig | cut -c-10 | uniq > $PREFIXES_OUTFILE
echo "Created '$PREFIXES_OUTFILE'."
echo ""
echo "Creating CSV file with all samples processed by TPP..."
$PYTHON2 scripts/wig_gb_to_csv.py -l $PREFIXES_OUTFILE -g $GENBANK_FILE -u $UNIQUE_FIELDS -f $FIELDS -o $CSV_OUTFILE
echo "Created '$CSV_OUTFILE'."
echo ""
(( TOTAL_RUN_TIME = SECONDS - INITIAL_START_TIME )) 
echo "********** TPP driver script finished in a total of $TOTAL_RUN_TIME seconds **********"

In [15]:
%%bash
#Run analysis script (depends on the file output/input.csv in order to run)

python3 scripts/hypersus_analysis.py 

JT test run time: 4.3813087940216064
JT test run time: 4.212145805358887
JT test run time: 4.130765914916992
JT test run time: 4.428714275360107
JT test run time: 4.321542978286743
JT test run time: 4.41239070892334
JT test run time: 4.46047306060791
JT test run time: 4.414608716964722


In [1]:
#Reorganize data for TableS1
infold = 'output/hypersus_analysis/'
outfold = 'output/hypersus_analysis/'

lfc_thresh = 0.5
padj_thresh = 0.05

import pandas as pd
import numpy as np
from math import log10, floor

def round_sig(x, sig=2):
    #Implement significant figures in python
    if np.isnan(x):
        return(np.nan)
    return round(x, sig-int(floor(log10(abs(x))))-1)

def make_tables1(infilename12h, infilename48h, outfilename, treatment, drug_dose_ls):
    #Merges tables output from analysis and changes formatting for disribution (TableS1)
    
    abx12_df = pd.read_csv(infilename12h).drop(['DMSO'],axis=1).rename({'pval-adj (BH)':'padj'},axis='columns')
    abx48_df = pd.read_csv(infilename48h).drop(['DMSO'],axis=1).rename({'pval-adj (BH)':'padj'},axis='columns')

    abx12_df[treatment+['pval','padj']] = abx12_df[treatment+['pval','padj']].applymap(round_sig)
    abx48_df[treatment+['pval','padj']] = abx48_df[treatment+['pval','padj']].applymap(round_sig)

    merged_df = abx12_df.merge(abx48_df, how='inner',on=['uid','product'],suffixes=('_12h','_48h'))
    merged_df['Prediction'] = 'Not Significant'
    hypersus_bool = ((merged_df['padj_12h'] < padj_thresh) & \
                (merged_df[treatment[-1]+'_12h'] < -lfc_thresh)) & \
                ((merged_df['padj_48h'] < padj_thresh) & \
                (merged_df[treatment[-1]+'_48h'] < -lfc_thresh))
    hypertol_bool = ((merged_df['padj_12h'] < padj_thresh) & \
                (merged_df[treatment[-1]+'_12h'] > lfc_thresh)) & \
                ((merged_df['padj_48h'] < padj_thresh) & \
                (merged_df[treatment[-1]+'_48h'] > lfc_thresh))

    merged_df = merged_df.rename({treatment[0]+'_12h':drug_dose_ls[0]+' - LFC_12h', \
                treatment[1]+'_12h':drug_dose_ls[1]+' - LFC_12h', \
                treatment[0]+'_48h':drug_dose_ls[0]+' - LFC_48h', \
                treatment[1]+'_48h':drug_dose_ls[1]+' - LFC_48h'}, axis=1)
    merged_df = merged_df.rename({'uid':'Genomic Feature','product':'Genbank Annotation'}, axis=1)

    merged_df['Prediction'][hypersus_bool]='Hypersusceptible'
    merged_df['Prediction'][hypertol_bool]='Hypertolerant'

    merged_df.to_csv(outfilename,index=False)

treatment = ['C++','C+++']
drug_dose_ls = ['Clarithromycin (0.54ug/mL)','Clarithromycin (5.4ug/mL)']
make_tables1(infold+'summary_12h_CLAR.csv', infold+'summary_48h_CLAR.csv', outfold+'TableS3_CLAR.csv', treatment, drug_dose_ls)
    
treatment = ['M+','M++']
drug_dose_ls = ['Moxifloxacin (0.1ug/mL)','Moxifloxacin (1.0ug/mL)']
make_tables1(infold+'summary_12h_MOXI.csv', infold+'summary_48h_MOXI.csv', outfold+'TableS4_MOXI.csv', treatment, drug_dose_ls)
    
treatment = ['R++','R+++']
drug_dose_ls = ['Rifabutin (0.063ug/mL)','Rifabutin (0.63ug/mL)']
make_tables1(infold+'summary_12h_RFB.csv', infold+'summary_48h_RFB.csv', outfold+'TableS5_RFB.csv', treatment, drug_dose_ls)
    
treatment = ['E+','E++']
drug_dose_ls = ['Ethambutol (0.21ug/mL)','Ethambutol (2.1ug/mL)']
make_tables1(infold+'summary_12h_EMB.csv', infold+'summary_48h_EMB.csv', outfold+'TableS6_EMB.csv', treatment, drug_dose_ls)

<ipython-input-1-ac7ce19a6481>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Prediction'][hypersus_bool]='Hypersusceptible'
<ipython-input-1-ac7ce19a6481>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Prediction'][hypertol_bool]='Hypertolerant'
<ipython-input-1-ac7ce19a6481>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Prediction'][hypersus_bool]='Hypersusceptible'
<ipython-input-1-ac7ce19a